# Torch implementation

The `PytorchClassifier` is a custom class that is part of the sklearn-wrap library, which is a library that allows you to use PyTorch models in the scikit-learn framework. Here is an example of how the PyTorchClassifier class can be defined:



- The `PyTorchClassifier` class is a simple implementation of a classifier that uses a PyTorch model. It has three main methods: `fit`, which trains the model on the given data, `predict`, which makes predictions on new data, and `score`, which computes the accuracy of the model on the given data. 
- The `fit` method takes the data as input and trains the model using the `create_model` function, the `batch_size`, and the `learning_rate` hyperparameters. 
- The `predict` method takes the data as input and makes predictions using the trained model. 
- The `score` method takes the data and the ground truth labels as input, and computes the accuracy of the model on the data.

In [31]:
class PyTorchClassifier:
  def __init__(self, create_model, batch_size=10, learning_rate=0.001, num_epochs=100):
    self.create_model = create_model
    self.batch_size = batch_size
    self.learning_rate = learning_rate
    self.num_epochs = num_epochs
    self.model = None
    self.criterion = None
    self.optimizer = None

  def fit(self, X, y):
    # Create the model, loss function, and optimizer
    self.model, self.criterion, self.optimizer = self.create_model(self.batch_size, self.learning_rate)

    # Convert the data to PyTorch tensors
    X = torch.Tensor(X)
    y = torch.Tensor(y).long()

    # Train the model
    for epoch in range(self.num_epochs):
      # Forward pass
      y_pred = self.model(X)

      # Compute the loss
      loss = self.criterion(y_pred, y)

      # Zero the gradients
      self.optimizer.zero_grad()

      # Backward pass
      loss.backward()

      # Update the weights
      self.optimizer.step()

  def predict(self, X):
    # Convert the data to PyTorch tensors
    X = torch.Tensor(X)

    # Forward pass
    y_pred = self.model(X)

    # Convert the predictions to numpy arrays
    return y_pred.detach().numpy()

  def score(self, X, y):
    # Make predictions
    y_pred = self.predict(X)

    # Compute the accuracy
    accuracy = (y_pred.argmax(axis=1) == y).mean()
    return accuracy

  def get_params(self, deep=True):
    return {
          'create_model': self.create_model,
          'batch_size': self.batch_size,
          'learning_rate': self.learning_rate,
          'num_epochs': self.num_epochs
   }

  def set_params(self, **params):
    self.create_model = params['create_model']
    self.batch_size = params['batch_size']
    self.learning_rate = params['learning_rate']
    self.num_epochs = params['num_epochs']
    return self



## `sklearn` interface for `GridSearchCV`
- In this code, we first use the `load_iris` function from the `sklearn` library to load the iris dataset into memory. We then standardize the features using the `StandardScaler` class from the `sklearn` library. 
- Next, we define the hyperparameters to be optimized, which are `batch_size` and `learning_rate`, as in the previous example. We then define the `create_model` function to create a simple model with two linear layers and one output layer. 
- The output layer uses the `Softmax` activation function since this is a multi-class classification problem. 
- We then create a `PyTorchClassifier` object and use the `GridSearchCV` function to find the optimal set of hyperparameters. Finally, we print the optimal hyperparameters using the `best_params_` attribute of the `GridSearchCV` object.

In [32]:
# Import necessary libraries
from sklearn.datasets import load_iris

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from torch.nn import Linear
import torch.nn as nn
import torch.optim as optim

# Load the iris dataset
X, y = load_iris(return_X_y=True)

# Standardize the features
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Define the hyperparameters to be optimized
batch_size = [10, 20, 40, 60, 80, 100]
epochs = [10, 50, 100] # Added based on KerasClassifier
learning_rate = [0.001, 0.01, 0.1, 0.2, 0.3]

# Define a function to create the model
def create_model(batch_size, learning_rate):
  model = nn.Sequential(
      Linear(4, 8),
      nn.ReLU(),
      Linear(8, 3),
      nn.Softmax()
  )
  criterion = nn.CrossEntropyLoss()
  optimizer = optim.Adam(model.parameters(), lr=learning_rate)
  return model, criterion, optimizer

# Create a PyTorch classifier
model = PyTorchClassifier(create_model, batch_size=batch_size, learning_rate=learning_rate)

# Create a Keras classifier
# ALT
# model = KerasClassifier(build_fn=create_model, verbose=0)
# Modify create_model with:
# def create_model():
  #model = Sequential()
  #model.add(Dense(12, input_dim=8, activation='relu'))
  #model.add(Dense(1, activation='sigmoid'))
  #model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
  #return model


# Define the parameter grid
param_grid = {'batch_size': batch_size, 'epochs': epochs, 'learning_rate': learning_rate}

# Use grid search to find the optimal set of hyperparameters
grid_search = GridSearchCV(model, param_grid=param_grid, cv=StratifiedKFold(n_splits=3))
# param_grid=dict(batch_size=batch_size, epochs=epochs, learning_rate=learning_rate)
grid_search.fit(X, y)

# Print the optimal set of hyperparameters
print(grid_search.best_params_)

TypeError: PyTorchClassifier.__init__() missing 1 required positional argument: 'create_model'


# Keras implementation:

In this code, we are using the `KerasClassifier` class from the Keras library to create a deep learning model. The `KerasClassifier` class takes a function that defines the architecture of the model as an argument, and we use the create_model function to create a simple model with two dense layers and one output layer. 
We are optimizing the hyperparameters `batch_size` and `epochs`, which control the size of the mini-batches used for training and the number of training iterations, respectively. The `GridSearchCV` function trains the model using different combinations of these hyperparameters, and then finds the combination that gives the best performance on the training data. The optimal hyperparameters are then printed using the `best_params_` attribute of the `GridSearchCV` object.

In [25]:
# Import necessary libraries
from sklearn.datasets import load_iris
from scikeras.wrappers import KerasClassifier, KerasRegressor
# TF Keras wrappers is deprecated - use above
# from keras.wrappers.scikit_learn import KerasClassifier
from keras.models import Sequential
from keras.layers import Dense

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler

# Load the iris dataset
X, y = load_iris(return_X_y=True)

# Standardize the features
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Define the hyperparameters to be optimized
batch_size = [10, 20, 40, 60, 80, 100]
epochs = [10, 50, 100] 
learning_rate = [0.001, 0.01, 0.1, 0.2, 0.3]

# Define a function to create the model
def create_model():
  model = Sequential()
  model.add(Dense(12, input_dim=4, activation='relu'))
  model.add(Dense(1, activation='sigmoid'))
  model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
  return model

# Create a PyTorch classifier
#model = KerasClassifier(build_fn=create_model, verbose=0) # build_fn is deprecated. Use `model`
model = KerasClassifier(model=create_model, verbose=0)

# Define the parameter grid
param_grid = {'batch_size': batch_size, 'epochs': epochs}

# Use grid search to find the optimal set of hyperparameters
grid_search = GridSearchCV(model, param_grid=param_grid, cv=StratifiedKFold(n_splits=3))
# param_grid=dict(batch_size=batch_size, epochs=epochs, learning_rate=learning_rate)
grid_search.fit(X, y)

# Print the optimal set of hyperparameters
print(grid_search.best_params_)

{'batch_size': 10, 'epochs': 10}


## Using RandomForest

In [14]:
# Import necessary libraries
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier

# Define the hyperparameters to be optimized
hyperparameters = {'n_estimators': [10, 100, 500, 1000],
                    'max_depth': [5, 10, 15, 20]}

# Create a random forest classifier with default parameters
clf = RandomForestClassifier()

# Use grid search to find the optimal set of hyperparameters
grid_search = GridSearchCV(clf, hyperparameters, cv=3)
grid_search.fit(X, y)

# Print the optimal set of hyperparameters
print(grid_search.best_params_)


{'max_depth': 5, 'n_estimators': 100}


# ResNet50 implementatation in PyTorch

In [34]:
import torch
import torch.nn as nn

class ResNet50(nn.Module):
    def __init__(self, num_classes=1000):
        super(ResNet50, self).__init__()

        self.conv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)

        self.layer1 = self._make_layer(64, 64, 3)
        self.layer2 = self._make_layer(64, 128, 4, stride=2)
        self.layer3 = self._make_layer(128, 256, 6, stride=2)
        self.layer4 = self._make_layer(256, 512, 3, stride=2)

        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512 * 4, num_classes)

    def _make_layer(self, in_channels, out_channels, num_blocks, stride=1):
        layers = []
        for i in range(num_blocks):
            if i == 0:
                layers.append(ResidualBlock(in_channels, out_channels, stride=stride))
            else:
                layers.append(ResidualBlock(out_channels, out_channels))
        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)

        return x

In [33]:
class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1):
        super(ResidualBlock, self).__init__()

        # Define the first convolutional layer, which has a kernel size of 3, stride of `stride`, and padding of 1
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False)
        # Define the batch normalization layer for the first convolutional layer
        self.bn1 = nn.BatchNorm2d(out_channels)
        # Define the ReLU activation function
        self.relu = nn.ReLU(inplace=True)
        # Define the second convolutional layer, which has the same configuration as the first layer
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False)
        # Define the batch normalization layer for the second convolutional layer
        self.bn2 = nn.BatchNorm2d(out_channels)

        # If the stride is not 1 or the number of input channels is not equal to the number of output channels,
        # a downsampling operation is performed to match the dimensions of the input and output tensors
        if stride != 1 or in_channels != out_channels:
            self.downsample = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(out_channels),
            )
        else:
            self.downsample = None

    def forward(self, x):
        # If a downsampling operation is performed, apply it to the input tensor
        if self.downsample is not None:
            residual = self.downsample(x)
        else:
            residual = x

        # Apply the first convolutional layer, batch normalization, and ReLU activation
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        # Apply the second convolutional layer and batch normalization
        x = self.conv2(x)
        x = self.bn2(x)

        # Add the residual tensor to the output of the second convolutional layer and apply the ReLU activation
        x += residual
        x = self.relu(x)

        return x
